# 1. dataset 

In [162]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time 
import joblib
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import font_manager

# sklearn 관련
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, Lasso
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
import ast 
import statsmodels.api as sm 
from time import time

# 전처리 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE

# lightgbm 관련
from lightgbm import LGBMRegressor
from lightgbm import plot_importance

In [163]:
# 한글 폰트 경로 설정
font_path = '/System/Library/Fonts/AppleSDGothicNeo.ttc'
font_name = font_manager.FontProperties(fname=font_path).get_name()
plt.rc('font', family=font_name)

In [164]:
data = pd.read_csv('combined_data.csv', encoding = "cp949")
data.head()
print(data.shape) #179,250

(179250, 27)


In [165]:
# Checking for null values
print(data.info())

# Checking for outliers
print(data.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179250 entries, 0 to 179249
Data columns (total 27 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   day_of_reg     179250 non-null  object 
 1   pick_rgn2_nm   179250 non-null  object 
 2   rider_cnt      179250 non-null  int64  
 3   order_cnt      179250 non-null  int64  
 4   datetime       179250 non-null  object 
 5   hour_reg       179250 non-null  int64  
 6   reg_date       179250 non-null  object 
 7   temp_c         179250 non-null  float64
 8   rain_c         179250 non-null  float64
 9   snow_c         179250 non-null  float64
 10  is_rain        179250 non-null  int64  
 11  month          179250 non-null  int64  
 12  week           179250 non-null  int64  
 13  is_holiday     179250 non-null  int64  
 14  q1             179250 non-null  float64
 15  q3             179250 non-null  float64
 16  IQR1.5         179250 non-null  float64
 17  outlier        179250 non-nul

In [166]:
data["datetime"] = pd.to_datetime(data["datetime"])
data["reg_date"] = pd.to_datetime(data["reg_date"])

data = data.sort_values(by="datetime")

In [167]:
data = data.drop(columns = ['rider_cnt', 'order_cnt','temp_c','rain_c', 
                            'snow_c', 'q1', 'q3', 'IQR1.5', 'outlier'])
print(data.head())

   day_of_reg pick_rgn2_nm            datetime  hour_reg   reg_date  is_rain   
0         토요일          강남구 2022-01-29 09:00:00         9 2022-01-29        0  \
24        토요일          중랑구 2022-01-29 09:00:00         9 2022-01-29        0   
23        토요일           중구 2022-01-29 09:00:00         9 2022-01-29        0   
22        토요일          종로구 2022-01-29 09:00:00         9 2022-01-29        0   
21        토요일          은평구 2022-01-29 09:00:00         9 2022-01-29        0   

    month  week  is_holiday  rider_cnt_2  rider_cnt_w_1  rider_cnt_w_2   
0       1     5           1        189.0            211            199  \
24      1     5           1         33.0             31             28   
23      1     5           1         27.0             29             26   
22      1     5           1         17.0             20             22   
21      1     5           1         54.0             62             56   

    rider_cnt_w_3  rider_cnt_w_4  order_cnt_w_1  order_cnt_w_2  order_cnt_

In [168]:
# data = data.dropna(subset=['rider_cnt_w_4'])
data.isna().sum()
print(data.shape) #179,250

(179250, 18)


In [169]:
# category  - pick_rgn2_nm, hour_reg, day_of_reg, is_rain, month, week, is_holiday
for col in ['pick_rgn2_nm', 'hour_reg', 'day_of_reg', 'is_rain', 'month', 'week', 'is_holiday'] : 
    data[col] = data[col].astype('category')

print(data.dtypes)


day_of_reg             category
pick_rgn2_nm           category
datetime         datetime64[ns]
hour_reg               category
reg_date         datetime64[ns]
is_rain                category
month                  category
week                   category
is_holiday             category
rider_cnt_2             float64
rider_cnt_w_1             int64
rider_cnt_w_2             int64
rider_cnt_w_3             int64
rider_cnt_w_4             int64
order_cnt_w_1             int64
order_cnt_w_2             int64
order_cnt_w_3             int64
order_cnt_w_4             int64
dtype: object


# 2. 데이터 전처리

In [ ]:
# numeric 변수 scale 
# scaler = StandardScaler()  #평균 0 , 분산 1로 조정
# #scaler = MinMaxScaler()

# num_vars = ['rider_cnt_2', 'rider_cnt_w_1', 'rider_cnt_w_2', 'rider_cnt_w_3',
#             'rider_cnt_w_4', 'order_cnt_w_1', 'order_cnt_w_2', 'order_cnt_w_3',
#             'order_cnt_w_4']
# data[num_vars] = scaler.fit_transform(data[num_vars])

# print(df.head(3))

## 2-1. one-hot-encoding

In [ ]:
# ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [3])], remainder='passthrough')
# X = np.array(ct.fit_transform(X))

In [170]:
var = ['pick_rgn2_nm', 'hour_reg','day_of_reg', 'is_rain', 'month','week','is_holiday']

encoder = OneHotEncoder()
onehot = pd.DataFrame(encoder.fit_transform(data[var]).toarray(), columns=encoder.get_feature_names_out(var), index = data.index)
df = pd.concat([onehot, data.drop(columns=var)], axis=1)
#print(df.head(3))
print(df.columns)

Index(['pick_rgn2_nm_강남구', 'pick_rgn2_nm_강동구', 'pick_rgn2_nm_강북구',
       'pick_rgn2_nm_강서구', 'pick_rgn2_nm_관악구', 'pick_rgn2_nm_광진구',
       'pick_rgn2_nm_구로구', 'pick_rgn2_nm_금천구', 'pick_rgn2_nm_노원구',
       'pick_rgn2_nm_도봉구', 'pick_rgn2_nm_동대문구', 'pick_rgn2_nm_동작구',
       'pick_rgn2_nm_마포구', 'pick_rgn2_nm_서대문구', 'pick_rgn2_nm_서초구',
       'pick_rgn2_nm_성동구', 'pick_rgn2_nm_성북구', 'pick_rgn2_nm_송파구',
       'pick_rgn2_nm_양천구', 'pick_rgn2_nm_영등포구', 'pick_rgn2_nm_용산구',
       'pick_rgn2_nm_은평구', 'pick_rgn2_nm_종로구', 'pick_rgn2_nm_중구',
       'pick_rgn2_nm_중랑구', 'hour_reg_9', 'hour_reg_10', 'hour_reg_11',
       'hour_reg_12', 'hour_reg_13', 'hour_reg_14', 'hour_reg_15',
       'hour_reg_16', 'hour_reg_17', 'hour_reg_18', 'hour_reg_19',
       'hour_reg_20', 'hour_reg_21', 'hour_reg_22', 'hour_reg_23',
       'day_of_reg_금요일', 'day_of_reg_목요일', 'day_of_reg_수요일', 'day_of_reg_월요일',
       'day_of_reg_일요일', 'day_of_reg_토요일', 'day_of_reg_화요일', 'is_rain_0',
       'is_rain_1', 'month_1', 'month

# 3. train, test set split

In [173]:
# train_ratio = 0.8
# total_samples = df.shape[0]
# train_samples = int(train_ratio * total_samples)
# df_train = df[:train_samples]
# df_test = df[train_samples:]

df_train = df[df["datetime"]<= '2023-02-14']
df_test = df[df["datetime"] >= '2023-02-15']

# print(df_train['reg_date'].min()) #2022-01-29
# print(df_test['reg_date'].min()) #2023-02-15

# print(df_train['reg_date'].max()) #2023-02-15
# print(df_test['reg_date'].max()) #2023-05-21

df_train = df_train.drop(columns = ['datetime', 'reg_date'])
df_test = df_test.drop(columns = ['datetime', 'reg_date'])
print(df_train.shape, df_test.shape) #142,875, 36,000

(142875, 77) (36000, 77)


In [174]:
# X_train, y_train 나누기
 
# X_train = train.iloc[:, :-1]
# y_train = df_train.iloc[:, -1]

# X_test = df_test.iloc[:, :-1]
# y_test = df_test.iloc[:, -1]

X_train = df_train.drop(columns=['rider_cnt_2'])
y_train = df_train['rider_cnt_2']

X_test = df_test.drop(columns=['rider_cnt_2'])
y_test = df_test['rider_cnt_2']

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(142875, 76) (142875,) (36000, 76) (36000,)


In [175]:
print(X_train.columns)

Index(['pick_rgn2_nm_강남구', 'pick_rgn2_nm_강동구', 'pick_rgn2_nm_강북구',
       'pick_rgn2_nm_강서구', 'pick_rgn2_nm_관악구', 'pick_rgn2_nm_광진구',
       'pick_rgn2_nm_구로구', 'pick_rgn2_nm_금천구', 'pick_rgn2_nm_노원구',
       'pick_rgn2_nm_도봉구', 'pick_rgn2_nm_동대문구', 'pick_rgn2_nm_동작구',
       'pick_rgn2_nm_마포구', 'pick_rgn2_nm_서대문구', 'pick_rgn2_nm_서초구',
       'pick_rgn2_nm_성동구', 'pick_rgn2_nm_성북구', 'pick_rgn2_nm_송파구',
       'pick_rgn2_nm_양천구', 'pick_rgn2_nm_영등포구', 'pick_rgn2_nm_용산구',
       'pick_rgn2_nm_은평구', 'pick_rgn2_nm_종로구', 'pick_rgn2_nm_중구',
       'pick_rgn2_nm_중랑구', 'hour_reg_9', 'hour_reg_10', 'hour_reg_11',
       'hour_reg_12', 'hour_reg_13', 'hour_reg_14', 'hour_reg_15',
       'hour_reg_16', 'hour_reg_17', 'hour_reg_18', 'hour_reg_19',
       'hour_reg_20', 'hour_reg_21', 'hour_reg_22', 'hour_reg_23',
       'day_of_reg_금요일', 'day_of_reg_목요일', 'day_of_reg_수요일', 'day_of_reg_월요일',
       'day_of_reg_일요일', 'day_of_reg_토요일', 'day_of_reg_화요일', 'is_rain_0',
       'is_rain_1', 'month_1', 'month

### numeric_scale 

In [189]:
# 입력 변수 
numeric_cols = ['rider_cnt_w_1', 'rider_cnt_w_2', 'rider_cnt_w_3',
                'rider_cnt_w_4', 'order_cnt_w_1', 'order_cnt_w_2', 'order_cnt_w_3',
                'order_cnt_w_4']

# scaler 
scaler_X = StandardScaler()

# X_train, X_test
X_train_scaled = scaler_X.fit_transform(X_train[numeric_cols])
X_test_scaled = scaler_X.transform(X_test[numeric_cols])

# 스케일링된 결과를 DataFrame으로 변환
X_train_scaled = pd.DataFrame(X_train_scaled, columns=numeric_cols, index = X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=numeric_cols, index = X_test.index)

# 원래의 범주형 변수들을 선택
categorical_cols = [col for col in X_train.columns if col not in numeric_cols]
X_train_cat = X_train[categorical_cols]
X_test_cat = X_test[categorical_cols]

# 스케일링된 DataFrame과 범주형 변수들을 병합
X_train_final = pd.concat([X_train_scaled, X_train_cat], axis=1)
X_test_final = pd.concat([X_test_scaled, X_test_cat], axis=1)



In [191]:
# 예측값
# y_train, y_test
scaler_y = StandardScaler()
y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1))
y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1))

# 3. regression - benchmark model

In [192]:
X_train_lm = sm.add_constant(X_train_final)

lr_1 = sm.OLS(y_train_scaled, X_train_lm).fit()

print(lr_1.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.973
Model:                            OLS   Adj. R-squared:                  0.973
Method:                 Least Squares   F-statistic:                 7.382e+04
Date:                Wed, 24 May 2023   Prob (F-statistic):               0.00
Time:                        16:08:35   Log-Likelihood:                 55577.
No. Observations:              142875   AIC:                        -1.110e+05
Df Residuals:                  142804   BIC:                        -1.103e+05
Df Model:                          70                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const              7.216e+08   1.11e+0

# 4.Machine Learning Modeling

## 4-1. 하이퍼파라미터 튜닝 - Grid Search 

### a. LightGBM model 

In [ ]:
classifier = LGBMRegressor()

parameters = [{'learning_rate': [0.1, 0.05, 0.01, 0.005], 'n_estimators': [50, 100, 200], 'max_depth': [3, 5, 7]},
              {'learning_rate': [0.15, 0.125, 0.1, 0.075], 'n_estimators': [50, 100, 200], 'max_depth': [3, 5, 7], 'num_leaves': [16, 32, 64]}]

grid_search = GridSearchCV(estimator=classifier,
                           param_grid=parameters,
                           scoring=['neg_mean_squared_error', 'neg_mean_absolute_error'],
                           cv=10,
                           n_jobs=-1,
                           refit='neg_mean_squared_error')                     

grid_search.fit(X_train, y_train)
best_rmse = np.sqrt(-1 * grid_search.cv_results_['mean_test_neg_mean_squared_error'][grid_search.best_index_])
best_mae = -1 * grid_search.cv_results_['mean_test_neg_mean_absolute_error'][grid_search.best_index_]
best_parameters = grid_search.best_params_
print("Best RMSE: {:.2f}".format(best_rmse))
print("Best MAE: {:.2f}".format(best_mae))
print("Best Parameters:", best_parameters)

# best rmse : 27.59
# best mae : 17.46
# Best Parameters: {'learning_rate': 0.075, 'max_depth': 7, 'n_estimators': 50, 'num_leaves': 64}

### b. ridge regression

In [ ]:
# Ridge Regression
ridge = Ridge()
ridge_param_grid = {'alpha': [0.1, 1.0, 2.0, 5.0, 10.0]}
ridge_grid_search = GridSearchCV(estimator=ridge, param_grid=ridge_param_grid, scoring='neg_mean_squared_error', cv=10, n_jobs=-1)
ridge_grid_search.fit(X_train, y_train)
print("Ridge Best RMSE: {:.2f}".format(np.sqrt(-ridge_grid_search.best_score_)))
print("Ridge Best Parameters: ", ridge_grid_search.best_params_)

# Ridge Best RMSE: 27.83
# Ridge Best Parameters:  {'alpha': 10.0}

### c. Lasso regression

In [ ]:
# Lasso Regression
lasso = Lasso(max_iter = 10000)
lasso_param_grid = {'alpha': [0.1, 1.0,2.0, 5.0, 10.0]}
lasso_grid_search = GridSearchCV(estimator=lasso, param_grid=lasso_param_grid, scoring='neg_mean_squared_error', cv=10, n_jobs=-1)
lasso_grid_search.fit(X_train, y_train)
print("Lasso Best RMSE: {:.2f}".format(np.sqrt(-lasso_grid_search.best_score_)))
print("Lasso Best Parameters: ", lasso_grid_search.best_params_)

# Lasso Best RMSE: 27.97
# Lasso Best Parameters:  {'alpha': 0.1}

### d. Support vector regressor

In [ ]:
# SVR
# svr = SVR()
# svr_param_grid = {'kernel': ['rbf', 'linear', 'poly', 'sigmoid'], 'C': [0.1, 1.0, 10.0], 'gamma': ['scale', 'auto']}
# svr_grid_search = GridSearchCV(estimator=svr, param_grid=svr_param_grid, scoring='neg_mean_squared_error', cv=10, n_jobs=-1)
# svr_grid_search.fit(df_X, df_y)
# print("SVR Best RMSE: {:.2f}".format(np.sqrt(-svr_grid_search.best_score_)))
# print("SVR Best Parameters: ", svr_grid_search.best_params_)

### e. Random Forest Regressor

In [ ]:
# Random Forest Regressor
rfr = RandomForestRegressor(random_state=0)
rfr_param_grid = {'n_estimators': [50, 100, 200], 'max_depth': [3, 5, 7, 9], 'min_samples_split': [2, 5, 10]}
rfr_grid_search = GridSearchCV(estimator=rfr, param_grid=rfr_param_grid, scoring='neg_mean_squared_error', cv=10, n_jobs=-1)
rfr_grid_search.fit(X_train, y_train)
print("Random Forest Regressor Best RMSE: {:.2f}".format(np.sqrt(-rfr_grid_search.best_score_)))
print("Random Forest Regressor Best Parameters: ", rfr_grid_search.best_params_)

# Random Forest Regressor Best RMSE: 25.89
# Random Forest Regressor Best Parameters:  {'max_depth': 7, 'min_samples_split': 5, 'n_estimators': 200}


### f. Decision Tree Regressor

In [ ]:

# Decision Tree Regressor
dtr = DecisionTreeRegressor(random_state=0)
dtr_param_grid = {'max_depth': [3, 5, 7], 'min_samples_split': [2, 5, 10]}
dtr_grid_search = GridSearchCV(estimator=dtr, param_grid=dtr_param_grid, scoring='neg_mean_squared_error', cv=10, n_jobs=-1)
dtr_grid_search.fit(X_train, y_train)
print("Decision Tree Regressor Best RMSE: {:.2f}".format(np.sqrt(-dtr_grid_search.best_score_)))
print("Decision Tree Regressor Best Parameters: ", dtr_grid_search.best_params_)

# Decision Tree Regressor Best RMSE: 27.85
# Decision Tree Regressor Best Parameters:  {'max_depth': 7, 'min_samples_split': 2}

## 4-2. train, test set 적용 

### a. train, test rmse, mae

In [180]:
def MAPE(y_test, y_pred):
    return np.mean(np.abs((y_test - y_pred) /y_test)) *100

In [197]:
y_train_scaled = y_train_scaled.ravel()
y_test_scaled=  y_test_scaled.ravel()

print(y_train_scaled.shape)
print(y_test_scaled.shape)

(142875,)
(36000,)


In [198]:
train_set = data[data["datetime"] <= '2023-02-14']
test_set = data[data["datetime"] >= '2023-02-15']

def execute_pipeline(X_train, y_train, X_test, y_test):
    regressors = [
        LinearRegression(),
        Ridge(alpha=10.0),
        Lasso(alpha=0.1, max_iter=5000),
        LGBMRegressor(learning_rate=0.075, max_depth=7, n_estimators=200, num_leaves=64),
        RandomForestRegressor(random_state=0, max_depth=7, min_samples_split=5, n_estimators=200),
        DecisionTreeRegressor(random_state=0, max_depth=7, min_samples_split=2)
    ]

    result_train = pd.DataFrame({'datetime': train_set["datetime"], 'day_of_reg': train_set["day_of_reg"],
                                 'pick_rgn2_nm': train_set["pick_rgn2_nm"], 'hour_reg': train_set["hour_reg"],
                                 'is_rain': train_set["is_rain"],
                                 'is_holiday': train_set["is_holiday"], 'y_test': train_set["rider_cnt_2"]})

    result_test = pd.DataFrame({'datetime': test_set["datetime"],
                                'pick_rgn2_nm': test_set["pick_rgn2_nm"], 'hour_reg': test_set["hour_reg"],
                                'is_rain': test_set["is_rain"], 'day_of_reg': test_set["day_of_reg"],
                                'is_holiday': test_set["is_holiday"], 'y_test': test_set["rider_cnt_2"]})

    scores = {}
    predictions = {}

    for reg in regressors:
        reg_name = reg.__class__.__name__
        scoring = {
            'rmse' : 'neg_root_mean_squared_error',
            'mae' : 'neg_mean_absolute_error',
            'r2' : 'r2'
        } 
        
        #cv_results = cross_validate(reg, X_train, y_train, cv = 5, scoring = scoring)
        
        reg.fit(X_train, y_train)
        y_pred_train = reg.predict(X_train)
        y_pred_test = reg.predict(X_test)
        
        y_pred_train_orig = scaler_y.inverse_transform(y_pred_train.reshape(-1, 1))
        y_pred_test_orig = scaler_y.inverse_transform(y_pred_test.reshape(-1, 1))
        
        y_train_orig = scaler_y.inverse_transform(y_train.reshape(-1, 1))
        y_test_orig = scaler_y.inverse_transform(y_test.reshape(-1, 1))
        
        rmse_train = np.sqrt(mean_squared_error(y_train_orig, y_pred_train_orig))
        mae_train = mean_absolute_error(y_train_orig, y_pred_train_orig)
        mape_train = MAPE(y_train_orig, y_pred_train_orig)
        rmse_test = np.sqrt(mean_squared_error(y_test_orig, y_pred_test_orig))
        mae_test = mean_absolute_error(y_test_orig, y_pred_test_orig)
        mape_test = MAPE(y_test_orig, y_pred_test_orig)
        r2_test = r2_score(y_test_orig, y_pred_test_orig)

        # 모델 저장
        model_file = f'model_{reg_name}.joblib'
        joblib.dump(reg, model_file)

        scores[reg_name] = {
            'Train RMSE': rmse_train,
            'Train MAE': mae_train,
            'Train MAPE': mape_train,
            'Test RMSE': rmse_test,
            'Test MAE': mae_test,
            'Test MAPE': mape_test,
            'Test R2': r2_test
        }
         
        result_train[f'y_pred_train_{reg_name}'] = y_pred_train_orig
        result_test[f'y_pred_test_{reg_name}'] = y_pred_test_orig
             
        predictions[reg_name] = y_pred_test_orig

    lasso_pred = predictions['Lasso']
    lgbm_pred = predictions['LGBMRegressor']
    rf_pred = predictions['RandomForestRegressor']
    average_pred = (lasso_pred + lgbm_pred + rf_pred) / 3

    result_test['y_pred_test_avg'] = average_pred

    scores_df = pd.DataFrame(scores).transpose()

    # train, test 예측치 저장
    result_train.to_csv('prediction_results_train_set.csv', index=False, encoding="cp949")
    result_test.to_csv('prediction_results_test_set.csv', index=False, encoding="cp949")

    return scores_df

scores_df = execute_pipeline(X_train_final, y_train_scaled, X_test_final, y_test_scaled)
print(scores_df)


                       Train RMSE  Train MAE  Train MAPE  Test RMSE   
LinearRegression        25.733961  16.782065    9.853570  22.870069  \
Ridge                   25.733730  16.779823    9.848112  22.838003   
Lasso                   31.669028  21.895493   15.320971  29.159710   
LGBMRegressor           19.114761  13.182057    7.604888  22.761059   
RandomForestRegressor   25.587065  16.569308    9.309451  22.151020   
DecisionTreeRegressor   27.151950  17.670834    9.896424  23.845591   

                        Test MAE  Test MAPE   Test R2  
LinearRegression       16.070583  10.131841  0.978731  
Ridge                  16.037885  10.100302  0.978791  
Lasso                  20.448937  14.344867  0.965424  
LGBMRegressor          15.290313   8.291173  0.978933  
RandomForestRegressor  14.879286   8.318566  0.980047  
DecisionTreeRegressor  15.967015   8.903370  0.976878  


### 세분화하여 MAE 값 확인

In [199]:
# 데이터 프레임에서 각 모델의 예측 값 - 실제 값 계산
predict = pd.read_csv('prediction_results_test_set.csv', encoding = "cp949")

predict['MAE_LinearRegression'] = abs(predict['y_pred_test_LinearRegression'] - predict['y_test'])
predict['MAE_Ridge'] = abs(predict['y_pred_test_Ridge'] - predict['y_test'])
predict['MAE_Lasso'] = abs(predict['y_pred_test_Lasso'] - predict['y_test'])
predict['MAE_LGBMRegressor'] = abs(predict['y_pred_test_LGBMRegressor'] - predict['y_test'])
predict['MAE_RandomForestRegressor'] = abs(predict['y_pred_test_RandomForestRegressor'] - predict['y_test'])
predict['MAE_DecisionTreeRegressor'] = abs(predict['y_pred_test_DecisionTreeRegressor'] - predict['y_test'])

# 'day_of_reg2', 'is_rain', 'is_holiday' 별로 차이의 평균 계산
result = predict.groupby(['is_rain', 'is_holiday']).agg({
    'MAE_LinearRegression': np.mean,
    'MAE_Ridge': np.mean,
    'MAE_Lasso': np.mean,
    'MAE_LGBMRegressor': np.mean,
    'MAE_RandomForestRegressor': np.mean,
    'MAE_DecisionTreeRegressor': np.mean
}).reset_index()

print(result)


   is_rain  is_holiday  MAE_LinearRegression  MAE_Ridge  MAE_Lasso   
0        0           0             15.543919  15.505295  19.552663  \
1        0           1             15.938566  15.918910  21.347851   
2        1           0             16.922143  16.855231  19.741049   
3        1           1             32.697768  32.693917  35.429030   

   MAE_LGBMRegressor  MAE_RandomForestRegressor  MAE_DecisionTreeRegressor  
0          14.339640                  13.609805                  14.622693  
1          15.856829                  16.166435                  17.421105  
2          15.412880                  16.147303                  17.560720  
3          35.651369                  33.921722                  34.680400  


### 변수 중요도 파악 

In [ ]:
lasso_model = joblib.load('model_Lasso.joblib')
lasso_coef = pd.Series(lasso_model.coef_, index=X_train.columns)
selected_feats_lasso = lasso_coef[lasso_coef!=0].index

print("Number of features selected by Lasso: ", len(selected_feats_lasso))
print("Features selected by Lasso: ", selected_feats_lasso)

# is_holiday2 제거
# day_of_reg 제거 
# specific_value 제거 
# Number of features selected by Lasso:  46
# Features selected by Lasso:  Index(['pick_rgn2_nm_강남구', 'pick_rgn2_nm_강동구', 'pick_rgn2_nm_강북구',
#        'pick_rgn2_nm_강서구', 'pick_rgn2_nm_광진구', 'pick_rgn2_nm_도봉구',
#        'pick_rgn2_nm_마포구', 'pick_rgn2_nm_서대문구', 'pick_rgn2_nm_서초구',
#        'pick_rgn2_nm_성북구', 'pick_rgn2_nm_영등포구', 'pick_rgn2_nm_은평구',
#        'pick_rgn2_nm_중랑구', 'hour_reg_9', 'hour_reg_12', 'hour_reg_17',
#        'hour_reg_18', 'hour_reg_19', 'hour_reg_20', 'hour_reg_22',
#        'hour_reg_23', 'day_of_reg_월목', 'is_rain_0', 'is_rain_1', 'month_2',
#        'month_3', 'month_4', 'month_5', 'month_6', 'month_7', 'month_9',
#        'month_10', 'week_2', 'week_3', 'week_4', 'day_of_reg2_월요일',
#        'is_holiday1_0', 'is_holiday1_1', 'rider_cnt_w_1', 'rider_cnt_w_2',
#        'rider_cnt_w_3', 'rider_cnt_w_4', 'order_cnt_w_1', 'order_cnt_w_2',
#        'order_cnt_w_3', 'order_cnt_w_4']



In [ ]:
def plot_feature_importances(model, model_name):
    importances = model.feature_importances_
    feat_names = X_train.columns
    feature_imp = pd.Series(importances, index=feat_names).sort_values(ascending=False)

    #print("Feature importances for ", model_name, " : ")
    #print(feature_imp)

    # Creating a bar plot
    plt.figure(figsize = (20,15))
    sns.barplot(x=feature_imp, y=feature_imp.index)
    # Add labels to your graph
    plt.xlabel('Feature Importance Score')
    plt.ylabel('Features')
    plt.title("Visualizing Important Features")
    plt.legend()
    plt.show()

lgbm_model = joblib.load('model_LGBMRegressor.joblib')
plot_feature_importances(lgbm_model, 'LGBMRegressor')

rf_model = joblib.load('model_RandomForestRegressor.joblib')
plot_feature_importances(rf_model, 'RandomForestRegressor')


## 다음주 6일 예측하기

In [ ]:
new_data = pd.read_csv('predict_data.csv', encoding = "cp949")
new_data.head()
print(new_data.shape) #2250,20

In [ ]:
print(new_data.info())

In [ ]:
new_data["reg_date"] = pd.to_datetime(data["reg_date"])
new_data = new_data.sort_values(by="reg_date")

In [ ]:
new_data = new_data.drop(columns = ['day_of_reg2', 'is_holiday'])
print(new_data.head())

In [ ]:
# category  - pick_rgn2_nm, hour_reg, day_of_reg, is_rain, month, week, is_holiday
for col in ['pick_rgn2_nm', 'hour_reg', 'day_of_reg', 'is_rain', 'month', 'week', 'is_holiday2'] : 
   new_data[col] = new_data[col].astype('category')

print(new_data.dtypes)

In [ ]:
# one-hot-encoding

var = ['pick_rgn2_nm', 'hour_reg','day_of_reg', 'is_rain', 'month','week','is_holiday2']

encode_data = new_data.sort_values(by="reg_date")
X_test = encode_data.drop(columns = ['reg_date'])

encoder = OneHotEncoder()
onehot = pd.DataFrame(encoder.fit_transform(encode_data[var]).toarray(), columns=encoder.get_feature_names_out(var), index = encode_data.index)
X_test = pd.concat([onehot, X_test.drop(columns=var)], axis=1)
#print(X_test.head(3))
print(X_test.columns)

 

### 존재하지 않는 변수 추가하기 (month,week)

In [ ]:
# 새로운 변수 생성
new_variables = ['month_1', 'month_2', 'month_3', 'month_4', 'month_6', 'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12', 'week_1', 'week_3', 'week_4', 'week_5', 'is_holiday2_1']

# 새로운 변수를 포함한 빈 DataFrame 생성
encode_data = pd.DataFrame(0, columns=new_variables, index= X_test.index)

# 기존 x_test DataFrame과 새로운 변수를 포함한 DataFrame을 병합
X_test = pd.concat([X_test, encode_data], axis=1)

# 결과 확인
#print(X_test.head())
print(X_test.columns)

### train/test set split 

In [ ]:
print(X_test.shape) # 2250,72

In [ ]:
X_train.isna().sum()

In [ ]:
desired_order = ['pick_rgn2_nm_강남구', 'pick_rgn2_nm_강동구', 'pick_rgn2_nm_강북구',
       'pick_rgn2_nm_강서구', 'pick_rgn2_nm_관악구', 'pick_rgn2_nm_광진구',
       'pick_rgn2_nm_구로구', 'pick_rgn2_nm_금천구', 'pick_rgn2_nm_노원구',
       'pick_rgn2_nm_도봉구', 'pick_rgn2_nm_동대문구', 'pick_rgn2_nm_동작구',
       'pick_rgn2_nm_마포구', 'pick_rgn2_nm_서대문구', 'pick_rgn2_nm_서초구',
       'pick_rgn2_nm_성동구', 'pick_rgn2_nm_성북구', 'pick_rgn2_nm_송파구',
       'pick_rgn2_nm_양천구', 'pick_rgn2_nm_영등포구', 'pick_rgn2_nm_용산구',
       'pick_rgn2_nm_은평구', 'pick_rgn2_nm_종로구', 'pick_rgn2_nm_중구',
       'pick_rgn2_nm_중랑구', 'hour_reg_9', 'hour_reg_10', 'hour_reg_11',
       'hour_reg_12', 'hour_reg_13', 'hour_reg_14', 'hour_reg_15',
       'hour_reg_16', 'hour_reg_17', 'hour_reg_18', 'hour_reg_19',
       'hour_reg_20', 'hour_reg_21', 'hour_reg_22', 'hour_reg_23',
       'day_of_reg_금', 'day_of_reg_월목', 'day_of_reg_주말', 'is_rain_0',
       'is_rain_1', 'month_1', 'month_2', 'month_3', 'month_4', 'month_5',
       'month_6', 'month_7', 'month_8', 'month_9', 'month_10', 'month_11',
       'month_12', 'week_1', 'week_2', 'week_3', 'week_4', 'week_5',
       'is_holiday2_0', 'is_holiday2_1', 'rider_cnt_w_1', 'rider_cnt_w_2',
       'rider_cnt_w_3', 'rider_cnt_w_4', 'order_cnt_w_1', 'order_cnt_w_2',
       'order_cnt_w_3', 'order_cnt_w_4']

X_test = X_test[desired_order]

In [ ]:

def get_predict(X_test):

    linear_model = joblib.load('model_LinearRegression.joblib')
    LGBM_model = joblib.load('model_LGBMRegressor.joblib')
    RF_model = joblib.load('model_RandomForestRegressor.joblib')

    result_df = pd.DataFrame({'reg_date': new_data["reg_date"],
                              'pick_rgn2_nm': new_data["pick_rgn2_nm"], 'hour_reg': new_data["hour_reg"],
                              'day_of_reg': new_data["day_of_reg"], 'is_rain': new_data["is_rain"],
                              'is_holiday': new_data["is_holiday2"]})

    y_pred_linear = linear_model.predict(X_test)
    y_pred_LGBM = LGBM_model.predict(X_test)
    y_pred_RF = RF_model.predict(X_test)
    
    result_df['y_pred_linear'] = y_pred_linear
    result_df['y_pred_LGBM'] = y_pred_LGBM
    result_df['y_pred_RF'] = y_pred_RF

    result_df.to_csv('prediction_results_latest6days.csv', index=False, encoding="cp949")


get_predict(X_test)